In [1]:
#Procesamiento de Lenguage Natural
#Taller #5: TF-IDF

In [2]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Punto 1: Pre-Procesamiento
#Leer el archivo Princesas.csv usando pandas y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [4]:
df = pd.read_csv('C:/Users\Estivens Reyes/archivos/Princesas.csv')
df = pd.DataFrame(df)
df

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [5]:
# Definir Función 
def preprocesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto.split()])
    return texto

In [6]:
# data procesada
df['preprocesado'] = df['Personalidad'].apply(lambda texto: preprocesado(texto))
df

,Princesa,Personalidad,preprocesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


In [7]:
#Punto 2: TF-IDF
#Crear la matriz TF-IDF

In [8]:
# tfidf_matrix
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(df['preprocesado'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_matrix.index = df["Princesa"].values

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


In [9]:
#Punto 3: Distancia del coseno
#3.1 Calcular la distancia del coseno entre cada una de las princesas

In [10]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818
Moana,0.951298,0.974925,0.965930,0.923546,0.968929,0.936091,0.945844,0.971443,0.971818,0.000000


In [11]:
#¿Cuáles son las princesas más parecidas?

In [50]:
for nombre in dist_cos.columns:
    df = dist_cos.sort_values(by = nombre, ascending = False)
        

In [51]:
print(f"La princesa {df[nombre][df[nombre] == df[nombre][0] ].index[0]} es la más parecida a " + nombre)


La princesa Cenicienta es la más parecida a Moana


In [48]:
#¿Cuáles son las princesas más diferentes?

In [49]:
for nombre in dist_cos.columns:
    print("\U0001f478\U0001f3fb " + dist_cos[nombre].idxmax() + " es la princesa más diferente a \U0001f478\U0001f3fc" + nombre )

👸🏻 Mérida es la princesa más diferente a 👸🏼Blancanieves
👸🏻 Moana es la princesa más diferente a 👸🏼Cenicienta
👸🏻 Moana es la princesa más diferente a 👸🏼Aurora
👸🏻 Tiana es la princesa más diferente a 👸🏼Bella
👸🏻 Mérida es la princesa más diferente a 👸🏼Jasmín
👸🏻 Mérida es la princesa más diferente a 👸🏼Pocahontas
👸🏻 Tiana es la princesa más diferente a 👸🏼Mulan
👸🏻 Mulan es la princesa más diferente a 👸🏼Tiana
👸🏻 Mulan es la princesa más diferente a 👸🏼Mérida
👸🏻 Cenicienta es la princesa más diferente a 👸🏼Moana
